In [1]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [2]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

corp = [x.split(" ") for x in corpus]
dct = Dictionary(corp)  

In [3]:
for id in dct:
    print(f"{id}: {dct[id]}")

0: document
1: first
2: is
3: the
4: this
5: second
6: and
7: one
8: third


In [64]:
c = [dct.doc2bow(line) for line in corp]  

model = TfidfModel(c)  

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 2), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(2, 1), (3, 1), (4, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]

In [50]:
"this is a test document".split(" ")

['this', 'is', 'a', 'test', 'document']

In [51]:
dct.doc2bow(['EXPERT', 'WARN', 'BACKLASH', 'DONALD', 'TRUMPS', 'CHINA', 'TRADE', 'POLICIES', 'NEW', 'YORK', 'TIMES'])

[]

In [69]:
[(dct[i[0]] + ": " + str(i[1])) for i in model.__getitem__(dct.doc2bow("this is a test document".split(" ")), eps=-1)]

['document: 1.0', 'is: 0.0', 'this: 0.0']

In [73]:
model.__getitem__(dct.doc2bow("this is a test document".split(" ")), eps=-1)

[(0, 1.0), (2, 0.0), (4, 0.0)]

In [7]:
[(dct[i[0]] + ": " + str(i[1])) for i in model.__getitem__(dct.doc2bow("this is test".split(" ")), eps=0)]

[]

In [8]:
[(dct[i[0]] + ": " + str(i[1])) for i in model[c[0]]]


['document: 0.383332888988391', 'first: 0.9236102512530996']

In [66]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import spacy
import re

class ElasticCorpus:
    
    def __init__(self, host, port, username, password):
        self.elastic = Elasticsearch([{'host':"localhost" , 'port': 9200}], http_auth=("elastic","elastic"))
        self.space = spacy.load('en_core_web_sm')
        self.dictonary = Dictionary()
        
    def __iter__(self):
        for entry in helpers.scan(self.elastic, query={"query": {"match_all": {}}}, _source=["title"], index="documents", size=2000):
            text = entry["_source"]["title"]
            text = re.sub('[^A-Za-z0-9 ]+', '', text)
            text = re.sub(' +', ' ', text)
            
            doc = self.space(text)
            tokens = [t.lemma_.upper() for t in doc if not t.is_stop]
            self.dictonary.add_documents([tokens])
            
            print(self.counter)
            
            yield self.dictonary.doc2bow(tokens)

In [67]:
ec = ElasticCorpus("localhost", 9200, "elastic", "elastic")

In [ ]:
model = TfidfModel(ec)  

In [76]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch([{'host':"localhost" , 'port': 9200}], http_auth=("elastic","elastic"))

i=0
for entry in helpers.scan(es, query={"query": {"match_all": {}}}, _source=["title"], index="documents", size=2000):
    i = i +1
    print(entry["_source"]["title"])
    break
    
print(i)

Experts Warn of Backlash in Donald Trump’s China Trade Policies - The New York Times
1


In [74]:
numList = [12, 13, 14, 15, 16]
numList[:3]

[12, 13, 14]

In [80]:
es.count(index='documents', body={'query': {"match_all": {}}})["count"]

50000